In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [ ]:
DIRECTORY = r"./Batch-1(0-9)/"
CATEGORIES = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
#"10", "11", "12", "13", "14", "15", "16", "17", "18",
#              "19", "20", "21", "22", "23", "24", "25", "26", "27",
#              "28", "29"

for category in  CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        plt.imshow(img_array, cmap='gray')
        plt.show()
        break
    break

In [ ]:
print(img_array.shape)

In [ ]:
data = []
for category in CATEGORIES:
    folder = os.path.join(DIRECTORY, category)
    class_labels = CATEGORIES.index(category)
    
    for img in os.listdir(folder):
        img = os.path.join(folder,img)
        img_array = cv2.imread(img)
        img_array = cv2.resize(img_array,(28,28))
        
        data.append([img_array, class_labels])

In [ ]:
training_data = []

def create_training_data():
        for category in  CATEGORIES:
            path = os.path.join(DIRECTORY, category)
            class_num = CATEGORIES.index(category)
            for img in os.listdir(path):
                try:
                    img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                    new_array = cv2.resize(img_array, (28, 28))
                    training_data.append([new_array, class_num])
                except Exception as e:
                    pass
create_training_data()

In [ ]:
print(len(training_data))

In [ ]:
import random
random.shuffle(training_data)

In [ ]:
for sample in training_data[:10]:
    print(sample[1])

In [ ]:
X = []
y = []

In [ ]:
for features, labels in training_data:
    X.append(features)
    y.append(labels)

X = np.array(X).reshape(-1, 28, 28, 1)

In [ ]:
# import pickle

# pickle_out = open("X.pickle", "wb")
# pickle.dump(X, pickle_out)
# pickle_out.close()

# pickle_out = open("y.pickle", "wb")
# pickle.dump(y, pickle_out)
# pickle_out.close()

In [ ]:
# pickle_in = open("X.pickle", "rb")
# X = pickle.load(pickle_in)

In [ ]:
# print(X[200], y[200])

In [ ]:
X = X/255

In [ ]:
X.shape

In [ ]:
X

In [ ]:
y = np.array(y)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation

In [ ]:
# X = pickle.load(open("X.pickle", "rb"))
# y = pickle.load(open("y.pickle", "rb"))



# model = Sequential()

# model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size = (2,2)))

# model.add(Conv2D(64, (3,3)))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size = (2,2)))

# model.add(Flatten())
# model.add(Dense(64))

# model.add(Dense(10))
# model.add(Activation('softmax'))

# model.compile(loss="categorical_crossentropy",
#               optimizer="adam",
#               metrics=['accuracy'])

# history = model.fit(X, y, batch_size = 32, validation_data = 0.1)

model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=X.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), input_shape=X.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(10, activation='softmax'))


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X,y, epochs=20, validation_split=0.1)

In [ ]:
import matplotlib.pylab as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.show()

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_float_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_float_model)
# Show model size in KBs.
float_model_size = len(tflite_float_model) / 1024
print('Float model size = %dKBs.' % float_model_size)

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

# Show model size in KBs.
quantized_model_size = len(tflite_quantized_model) / 1024
print('Quantized model size = %dKBs,' % quantized_model_size)
print('which is about %d%% of the float model size.'\
      % (quantized_model_size * 100 / float_model_size))

In [ ]:
from tkinter.filedialog import SaveAs


f = open('mnist.tflite', "wb")
f.write(tflite_quantized_model)
f.close()

SaveAs('mnist.tflite')

print('`mnist.tflite` has been saved!')